In [12]:
print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa".replace("a", "3"))

33333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333
